# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA 
* Model: GPT-2
* Evaluation approach: train.eval
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [36]:
from transformers import AutoModelForCausalLM

model_name="gpt2"

model = AutoModelForCausalLM.from_pretrained(model_name)

In [93]:
from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
dataset = {split: dataset for split, dataset in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    dataset[split] = dataset[split].shuffle(seed=57).select(range(500))

# Show the dataset
dataset

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True,padding="max_length"), batched=True)

# Inspect the available columns in the dataset
tokenized_dataset["train"]

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [38]:
dataset['train'][0]

{'text': 'In the future of 1985, a governmental committee headed by Howard Hesseman, is holding hearings on TV\'s first uncensored network. They sample it\'s programming, that play as a series of skits. I can name the good \'skit\' movies on one hand, not using my thumb. "Amazon Women on the Moon", "Kentucky Fried Movie", "The Meaning of Life", and "Mr. Mike\'s Mondo Video". Notice how I didn\'t mention "Tunnel Vision"? The reason for that is that this \'movie\' is death in cinematic form. None of the skits are even remotely funny, or even the least bit clever. It takes some sort of great ineptitude on the film makers\' part to not even get one laugh out of me.<br /><br />My Grade: F <br /><br />Eye Candy: Dody Dorn goes full frontal',
 'label': 0}

In [94]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True,padding="max_length"), batched=True)

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.config.pad_token_id = model.config.eos_token_id

for param in model.parameters():
    param.requires_grad = True

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/movie_sentiment",
        # Set the learning rate
        learning_rate=2e-3,
        # Set the per device train batch size and eval batch size
        per_device_eval_batch_size=4,
        per_device_train_batch_size=4,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693363,0.512000
2,No log,0.801775,0.634000


TrainOutput(global_step=250, training_loss=0.8084098510742187, metrics={'train_runtime': 356.6847, 'train_samples_per_second': 2.804, 'train_steps_per_second': 0.701, 'total_flos': 522593501184000.0, 'train_loss': 0.8084098510742187, 'epoch': 2.0})

In [7]:
trainer.evaluate()

{'eval_loss': 0.6933625340461731,
 'eval_accuracy': 0.512,
 'eval_runtime': 44.7458,
 'eval_samples_per_second': 11.174,
 'eval_steps_per_second': 2.794,
 'epoch': 2.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [19]:
from peft import LoraConfig
config = LoraConfig()

In [20]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [21]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [22]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.23643136409814364


In [23]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [74]:
from peft import AutoPeftModelForCausalLM
lora_model = AutoPeftModelForCausalLM.from_pretrained("gpt-lora")

In [75]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
inputs = tokenizer("Hello, my name is ", return_tensors="pt")


In [76]:
inputs

{'input_ids': tensor([[15496,    11,   616,  1438,   318,   220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [27]:
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(tokenizer.batch_decode(outputs))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Hello, my name is _____. I am a student at the University of']


In [103]:
def compute_metrics2(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # For accuracy
    accuracy = accuracy_score(labels, predictions)
    # For precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [104]:

#---------------------------------------------------------
# I have no idea how to make this work I tried everything 
# and I dont know what is wrong with it
#---------------------------------------------------------

from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from transformers import AutoModelForCausalLM

model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

lora_model = get_peft_model(model, config)
lora_model.config.pad_token_id = model.config.eos_token_id


trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/movie_sentiment2",
        # Set the learning rate
        learning_rate=2e-3,
        # Set the per device train batch size and eval batch size
        per_device_eval_batch_size=1,
        per_device_train_batch_size=1,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics2,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [100]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


AttributeError: 'dict' object has no attribute 'map'